In [1]:
import numpy as np
import torch
import copy
from scipy.integrate import solve_ivp
from scipy.special import softmax
import matplotlib.pyplot as plt
import scipy.stats
import scipy.linalg

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
#np.core.arrayprint._line_width = 80

np.set_printoptions(edgeitems=30, linewidth=250, 
    formatter=dict(float=lambda x: "% .4f" % x))

### Useful functions

In [2]:
# Useful fuctions 
def Euclid(x, y):
    '''
        d = Euclid(x, y) 
        Computes the Euclidean distance between the two vectors x and y 
        x and y are arrays of real numbers
        d is the Euclidean distance between the two arrays
    '''
    ans = np.sqrt( np.sum( np.square(x-y) ) )
    return(ans)

def TensEuclid(x, y):
    '''
    d = TensEuclid(x, y)
    Computes the average Euclidean distance between all vectors in a tensor 
    x and y are tensors of real numbers
    d is the average Euclidean distance between the vectors in the tensor (like MSE but with sqrt)
    used to get an idea of error, or how far off each vector is from its target
    '''
    
    n = len(x)
    if n != len(y):
        print("input tensors must be of same length")
    else:
        err = []
        for i in range(n):
            err.append(Euclid(x[i], y[i]))
    return sum(err)/n

In [3]:
# Useful functions
def IsScalar(x):
    if type(x) in (list, np.ndarray,):
        return False
    else:
        return True

def Thresh(x):
    if IsScalar(x):
        val = 1 if x>0 else -1
    else:
        val = np.ones_like(x)
        val[x<0] = -1.
    return val

def ham(X, Y):
    mul = np.multiply(X, Y)
    H = 0
    for i in mul:
        if i<=0:
            H=H+1
            
    return H
    
def Hamming(X, Y):
    '''
     H = Hamming(X, Y)
     
     Computes the Hamming distance between rows of X and Y.
     If X and/or Y have multiple rows, then the output will
     be an array.
     
     Inputs:
      X   (M,D) array
      Y   (N,D) array
      
     Output:
      H   (M,N) array
    '''
    H = np.zeros((len(X), len(Y)), dtype=int)
    for i,x in enumerate(X):
        for j,y in enumerate(Y):
            H[i,j] = np.sum(x*y<=0.)
    return H

def Perturb(x, p=0., n=0):
    '''
        y = Perturb(x, p=0., n=0)
        
        Apply binary noise to x. With probability p, each bit will be randomly
        set to -1 or 1.
        
        Inputs:
          x is an array of binary vectors of {-1,1}
          p is the probability of each bit being randomly flipped (n=0)
          n is the number of bits to flip (p=0)
        
        Output:
          y is an array of binary vectors of {-1,1}
    '''
    y = copy.deepcopy(x)
    for yy in y:
        if p!=0.:
            for k in range(len(yy)):
                if np.random.rand()<p:
                    yy[k] = Thresh(np.random.randint(2)*2-1)
        if n>0:
            k = np.random.randint(len(yy))
            yy[k] *= -1
    return y

def lse(beta, x):
    val = np.sum(np.exp(beta*x))
    return np.log(val) / beta


def sigmoid(x):
    return 1/( 1 + np.exp(-x) )


def relu(x):
    return np.maximum(x, 0)


def lif(x):
    x_out = [0 if xi <= 1 else (1/( 0.5 - np.log( 1 - (1/xi)) )) for xi in x]
    return x_out
 

def V_train(X, N_batch):
    ''' 
    X = self.Zeta ('correct dataset of patterns')
    N_e = dimesion of encoded hidden layer 
    
        returns a dataset to use for training decoding weights. 
        the dataset will be of size (N_samples, N_v) where N_v is the length of one pattern vector 
        it will include multiple copies of X as well as perturbed versions of it
        (usually N_samples is "a magnitide larger than N_e" 
    '''
    Nh, Nv = np.shape(X)    
    V_t = X
    for i in range( int( 0.80*(N_batch-1) ) ):
        V_t = np.vstack((V_t, X))
    for i in range(int( 0.20*(N_batch) ) ):
        V_t = np.vstack( (V_t, X + np.random.normal(0, 0.2, size=X.shape)) )          
    #print('samples:', np.shape(V_t)[0])
    return V_t    


#Computes the magnitudes of the rows in a matrix and returns the maximum and average magnitudes
def max_mag(x):
    samples, bits = x.shape
    mags = []
    for i in range(samples):
        mag = np.sqrt(np.sum(np.square(x[i])))
        mags.append(mag)
    #print('Magnitudes:', mags)
    max_mag = np.max(mags)
    avg_mag = np.average(mags)
    return max_mag, avg_mag


def plot_state(t, v, I):
    Nv = v.shape[1]
    t0, tf = t[0], t[-1]
    II = np.vstack([I,I]).T
    plt.plot([t0, tf], II.T, ':')
    plt.plot(t, v);

### Original LSE Network

In [4]:
# Simple version
class LSEHopfield(object):
    '''
     net = ExpHopfield(Zeta)
     
     Creates a Hopfield network that uses LSE energy.
     Zeta holds the target patterns in rows.
    '''
    def __init__(self, Zeta, beta=1., tau=1., kappa=1., decaytype="Off", split_time=1.5):
        self.Zeta = copy.deepcopy(Zeta)
        self.Nh, self.Nv = self.Zeta.shape
        self.beta = beta
        self.kappa = kappa
        self.tau = tau
        self.decaytype = decaytype
        self.split_time = split_time
        
    def dynamics(self, I):
        '''
         f = net.dynamics(I)

         Returns a dynamics function of the form
           f(t, z)
         that can be used in the IVP solvers.
         
         I is the input to the network.
        '''
        
        def func(t, z):
            '''
             z = [v, h]
             v, h, f are all N-D. b and c are scalar
            '''
            v = z[               :self.Nv        ]
            h = z[self.Nv        :self.Nv+self.Nh]
            
            # Changing I to different targets 
            self.I = I
            if len(self.I) == 2:
                if t < self.split_time:
                    self.I = self.I[0]
                if t >= self.split_time:
                    self.I = self.I[1]
            dh = ( self.Zeta@v - h ) / self.tau
            
            #No decay of I 
            if self.decaytype == "Off":
                if t <= self.split_time: 
                    Beta = 0.5 
                if t > self.split_time:
                    Beta = 0
                dv = ( (1-Beta)*self.Zeta.T@softmax(h) - self.kappa*v + Beta*self.I ) / self.tau 
            
            #Decay of I
            if self.decaytype == "Linear":
                dv = ( self.Zeta.T@softmax(h) - self.kappa*v + self.I*(1-self.beta*t) ) / self.tau  
            if self.decaytype == "Expo":
                Beta = np.exp(-self.beta*t)
                dv =  ( (1-Beta)*self.Zeta.T@softmax(h) - self.kappa*v + self.I*(Beta) ) / self.tau
 
            dzdt = np.concatenate((dv, dh))
            return dzdt
        return func

### BIO LSE Network (no learning, no encoding)

In [5]:
class BioLSEHopfield(object):
    '''
     net = ExpHopfield(Zeta, tau=1, tau_sm=1, kappa=1)
     
     Creates a Hopfield network that uses LSE energy and biologically plausible implementation of Softmax.
      
      Zeta holds the target patterns in rows.
      tau = time constant for the v and h nodes.
      tau_sm = time constant for the f and c nodes (for computing softmax)
      (tau_sm < tau)
      kappa is the decay coefficient for v
    '''
    def __init__(self, Zeta, beta=1., tau_v=1., tau_h=1., tau_sm=1., kappa=1., decaytype="Off", T=1):
        self.Zeta = copy.deepcopy(Zeta)
        self.Nh, self.Nv = self.Zeta.shape
        self.tau_v = tau_v
        self.tau_h = tau_h
        self.tau_sm = tau_sm
        self.kappa = kappa
        self.beta = beta
        self.decaytype = decaytype
        self.T = T
        
    def dynamics(self, I):
        '''
         f = net.dynamics(I)

         Returns a dynamics function of the form
           f(t, z)
         that can be used in the IVP solvers.
         
         I is the input to the network.
        '''
        
        def func(t, z):
            '''
             z = [v, h, f, b]
             v, h, f are all N-D. b is a scalar
            '''
            v = z[               :self.Nv        ]
            h = z[self.Nv        :self.Nv+self.Nh]
            f = z[self.Nv+self.Nh:-1             ]
            c = z[-1]

            T = self.T
            
            
            #No decay of I 
            if self.decaytype == "Off":
                if t < T:
                    Beta = 0.5
                if t>= T:
                    Beta = 0
                dv = ( (1-Beta)*self.Zeta.T@np.exp(f) - self.kappa*v + Beta*I ) / self.tau_v
                #dv = ( self.Zeta.T@softmax(h) - v + I ) / self.tau
            
            #Exponential decay of 
            if self.decaytype == "Expo":
                Beta = np.exp(-self.beta*t)
                dv = ( (1-Beta)*self.Zeta.T@np.exp(f) - self.kappa*v + I*(Beta) ) / self.tau_v
            
            
            dh = ( self.Zeta@v - h ) / self.tau_h
            df = ( h - c - f ) / self.tau_sm
            dc = ( np.log(np.sum(np.exp(h))) - c ) / self.tau_sm

            dzdt = np.concatenate((dv, dh, df, [dc]))
            
            return dzdt
        
        return func

### BIO LSE Network (with learning, no encoding)

In [6]:
class BioLSEHopfield_Learn(object):
    '''
     net = BioLSEHopfield_Learn(Zeta, beta, tau_w=1, tau_h=1, tau_sm=1, tau_I=1, kappa=1, decaytype, batchsize)
     
      Creates a Hopfield network that uses LSE energy and biologically plausible implementation of Softmax.
      
      Option to use network to learn weights (training=True) or use network normally (training=False)
      
      Zeta holds the target patterns in rows.
      beta is the decay constant for the input (to have input decay set decaytype='Expo')
      tau_w = time constant for weights or for the v nodes when training is off)
      tau_h = time constant for h nodes
      tau_sm = time constant for the f and c nodes (for computing softmax) (tau_sm < tau_h <tau_w)
      kappa is the decay coefficient for v
      decay_type (of input) can be "Off" means no decay, decay_type = "Expo" means exponential decay 
      
      If you want to use the network for sample, set batchsize=1. Batchsize defaults to the entire dataset.
    '''
    def __init__(self, Zeta, beta=1., tau_w=1., tau_h=1., tau_sm=1., t_I=1., kappa=1., decaytype="Off", batchsize=None):
        self.Zeta = copy.deepcopy(Zeta)
        self.Nh, self.Nv = self.Zeta.shape
        if batchsize is None:
            self.bs = self.Nh
        else:
            self.bs = 1
        self.tau_w = tau_w
        self.tau_h = tau_h
        self.tau_sm = tau_sm
        self.t_I = t_I
        self.kappa = kappa
        self.beta = beta
        self.decaytype = decaytype
        self.I = 0.    # This is just here to address the presence of I with Beta
    
    def pack(self, v, h, f, c, X):
        '''
         z = net.pack(v, h, f, c, X)
         Can be undone using v,h,f,c,X = net.unpack(z).
        '''
        if self.bs==1:
            
            X=X.ravel().reshape((self.Nh*self.Nv, 1))
            return np.concatenate((v,h,f,c.T,X), axis=0).ravel()
        else:
            return np.concatenate((v,h,f,c.T,X), axis=0).ravel()
    
    def unpack(self, z):
        '''
         v,h,f,c,X = net.unpack(z)
         Can be undone using net.pack(v, h, f, c, X).
        '''
        Nh = self.Nh
        Nv = self.Nv
        bs = self.bs
        v = z[ : Nv*bs]
        hstart = len(v)
        h = z[hstart : hstart + Nh*bs]
        fstart = hstart + len(h)
        f = z[fstart : fstart + Nh*bs]
        cstart = fstart + len(f)
        c = z[cstart : cstart + bs]
        Xstart = cstart + len(c)
        X = z[Xstart :]

        v = v.reshape( (Nv, bs) )
        h = h.reshape( (Nh, bs) )
        f = f.reshape( (Nh, bs) )
        X = X.reshape( (Nv, Nh) )
        c = c.reshape( (bs, 1 ) )
        
        return v, h, f, c, X
    
    def dynamics(self, t, z, training):
        '''
         z = [v, h, f, c, X]

         During training weights (training = True):
         dv and dh are set to zero (so v and h do not change)
         v and h should be set to the targets and 1-hot respectively by setting their components
         in the ititial z array (y_0) to these values and setting their derivative to zero
         f, c, X, all follow the dynamical equations below.
         @equilibrium, f = h - LSE(h) (Nh x Nh),  c = LSE(h) (Nh x 1), X = Zeta.T (Nh x Nv)


         During testing (training = False):
         dX is set to zero (so X will not change)
         X is fixed to be the weights by setting initial z array (y_0) to these values and setting dXdt to zero.
         v, h, f, c all follow their dynamical equations.
         @equilibrium, f = h - LSE(h) (Nh x Nh),  c = LSE(h) (Nh x 1), v = (1/kappa)*(Zeta.T softmax(h) + I) 
        '''
        v,h,f,c,X = self.unpack(z)        

        Nh = self.Nh
        Nv = self.Nv
        bs = self.bs
        
        if training:
            #if we want to learn the weights, clamp v and h by setting initial values to targets and 1-hot and dvdt=dhdt=0. 
            dv = np.zeros_like(v)
            dh = np.zeros_like(h)
            df = ( h - c.T - f) / self.tau_sm
            dc = ( np.log(np.sum(np.exp(h), axis=0, keepdims=True)).T - c ) / self.tau_sm
            dX = ( v @ np.exp(f) - X )/ self.tau_w
        
        else:
            #if we want to actually use the network, we clamp X to the weights by setting this in the initial values and dXdt=0.
            #this means we will take 1 target at a time, so the dimensions reduce here
            
            dX = np.zeros_like(X)
           
        
            #### Decay Equations ####
            #No decay
            #if self.decaytype == "Off":
                #dv = ( X@np.exp(f) - self.kappa*v + self.I ) / self.tau_w
            #Exponential decay
            #if self.decaytype == "Expo":
                #Beta = np.exp(-self.beta*t)
                #dv = ( (1-Beta)*X@np.exp(f) - self.kappa*v + self.I*(Beta) ) / self.tau_w
                
            if t<self.t_I: 
                I = self.I
                Beta=0.5
            if t>=self.t_I:
                I = 0
                Beta = 0
    
            dv = ( (1-Beta)*X@np.exp(f) - self.kappa*v + I*(Beta) ) / self.tau_w                
            dh = ( X.T@v - h ) / self.tau_h
            df = ( h - c.T - f) / self.tau_sm
            dc = (( np.log(np.sum(np.exp(h), axis=0))).reshape((bs,1)) - c ) / self.tau_sm
            
        dzdt = self.pack(dv, dh, df, dc, dX)
        
        return dzdt

### BIO LSE Network (no learning, with encoded hidden representation) 

In [7]:
class BioLSEHopfield_Encode(object):
    '''
     net = BioLSEHopfield_Encode()
     
      Creates a Hopfield network that uses LSE energy and biologically plausible implementation of Softmax 
      and a higher dimensional hidden representation.
      
      Nh_e = Dimensions of encoded hidden layer
      enc_w = 'binary' or 'real' sets the encoding weights to be either binary values or real values respectively. 
      N_samp = Number of target samples in training dataset for decoding weights
    '''
    def __init__(self, Zeta, Nh_e, beta=1., tau_w=1., tau_h=1., tau_sm=1., t_I =1., kappa=1., decaytype="Off", enc_w = 'binary', N_batch=1, batchsize=None):
        
        self.Zeta = copy.deepcopy(Zeta)       
        self.Nh, self.Nv = self.Zeta.shape
        self.Nh_e = Nh_e 
        self.N_batch = N_batch
        if batchsize is None:
            self.bs = self.Nh
        else:
            self.bs = 1
        self.tau_w = tau_w
        self.tau_h = tau_h
        self.tau_sm = tau_sm
        self.t_I = t_I
        self.kappa = kappa
        self.beta = beta
        self.decaytype = decaytype
        self.I = 1    
        self.D_h = 1.
        self.D_LSEh=1.
        self.enc_w = enc_w
        self.drop = np.random.choice(self.Nh_e, int(0.75*self.Nh_e), replace=False) #for dropping a certain amount of neurons in the encoded hidden layer
        
        
        
        ### ENCODING & LEARNING DECODERS###
        
        #Create samples to train decoders with 
        V = V_train(self.Zeta, self.N_batch)
        
        #Encoding weights & bias (binary)
        h_rad = max_mag(XX@XX.T)[0]
        if self.enc_w == 'binary':
            self.W_e = h_rad*(2*(np.random.binomial(n=1, p=0.5, size=(self.Nh, self.Nh_e))) - 1) 
            self.B_e = 2*(np.random.binomial(n=1, p=0.5, size=(self.Nh_e,))) - 1 
        #Encoding weights & bias (real)
        if self.enc_w == 'real':
            self.W_e = self.Nv*np.sqrt(self.Nh)*np.random.normal(0, 1, size=(self.Nh, self.Nh_e))               
            self.B_e = np.random.normal(0, 1, size=(self.Nh_e,))      

        #Activities of hidden layer
        Ah = V@self.Zeta.T
        
        LSE_Ah = np.log( np.sum( np.exp(Ah), axis=1 ) )

        
        #Project/encode the hidden layer
        Acth = sigmoid(Ah@self.W_e + self.B_e)

        #Want to decode A and LSE(A) from Act (learn the decoders)
        self.D_h = np.linalg.lstsq(Acth, Ah, rcond=None)[0].T
        self.D_LSEh = np.linalg.lstsq(Acth, LSE_Ah, rcond=None)[0].T

            
    def pack(self, v, h, f, c, X):
        '''
         z = net.pack(v, h, f, c, X)
         Can be undone using v,h,f,c,X = net.unpack(z).
        '''
        if self.bs==1:
            
            X=X.ravel().reshape((self.Nh*self.Nv, 1))
            return np.concatenate((v,h,f,c.T,X), axis=0).ravel()
        else:
            return np.concatenate((v,h,f,c.T,X), axis=0).ravel()
    
    def unpack(self, z):
        '''
         v,h,f,c,X = net.unpack(z)
         Can be undone using net.pack(v, h, f, c, X).
        '''
        Nh = self.Nh
        Nv = self.Nv
        bs = self.bs
        v = z[ : Nv*bs]
        hstart = len(v)
        h = z[hstart : hstart + Nh*bs]
        fstart = hstart + len(h)
        f = z[fstart : fstart + Nh*bs]
        cstart = fstart + len(f)
        c = z[cstart : cstart + bs]
        Xstart = cstart + len(c)
        X = z[Xstart :]

        v = v.reshape( (Nv, bs) )
        h = h.reshape( (Nh, bs) )
        f = f.reshape( (Nh, bs) )
        X = X.reshape( (Nv, Nh) )
        c = c.reshape( (bs, 1 ) )
        
        return v, h, f, c, X
    
    def dynamics(self, I):
        '''
         f = net.dynamics(I)

         Returns a dynamics function of the form
           f(t, z)
         that can be used in the IVP solvers.
         
         I is the input to the network.
        '''
        self.I = I
        
        def func(t, z):
            '''
             z = [v, h_e, f, c]
             v, h, f are all N-D. b and c are scalar
            '''

            v =   z[                 : self.Nv        ]
            h_e = z[self.Nv          : self.Nv+self.Nh_e]
            f =   z[self.Nv+self.Nh_e: self.Nv+self.Nh_e+self.Nh]
            c =   z[-1]
            
            
            dh_e = ( sigmoid( self.W_e.T@(self.Zeta@v) + self.B_e ) - h_e ) / self.tau_h
            
            #uncomment below to turn off of the neurons in the projected hidden layer (randomly selected)
            #dh_e=[0 if ind in self.drop else x for ind, x in enumerate(dh_e)]
            
            if t<self.t_I: 
                self.I = self.I
                Beta=1
            if t>=self.t_I:
                self.I = 0
                Beta = 0
                
            dv = ( (1-Beta)*self.Zeta.T@np.exp(f) - self.kappa*v + self.I*(Beta) ) / self.tau_w    
            
            dc = ( self.D_LSEh@h_e - c )/self.tau_sm
            
            df = ( self.D_h@h_e - c - f ) / self.tau_sm
             
            dzdt = np.concatenate((dv, dh_e, df, [dc]))
            return dzdt
        
        return func

### BIO LSE HN with encoded hidden representation and learning

In [8]:
class BioLSE_Encode_Learn(object):
    '''
     net = BioLSE_Encode_Learn()
     
      Creates a Hopfield network that uses LSE energy and biologically plausible implementation of Softmax 
      and a higher dimensional hidden representation.
      
      Nh_e = Dimensions of encoded hidden layer
      enc_w = 'binary' or 'real' sets the encoding weights to be either binary values or real values respectively. 
      N_samp = Number of target samples in training dataset for decoding weights
      
      Option to use network to learn weights (training=True) or use network normally (training=False)
      
    '''
    def __init__(self, Zeta, Nh_e, beta=1., tau_w=1., tau_h=1., tau_sm=1., t_I =1., kappa=1., decaytype="Off", enc_w = 'binary', N_batch=1, batchsize=None):
        
        self.Zeta = Zeta      
        self.Nh, self.Nv = self.Zeta.shape
        self.Nh_e = Nh_e 
        self.N_batch = N_batch
        if batchsize is None:
            self.bs = self.Nh
        else:
            self.bs = 1
        self.tau_w = tau_w
        self.tau_h = tau_h
        self.tau_sm = tau_sm
        self.t_I = t_I
        self.kappa = kappa
        self.beta = beta
        self.decaytype = decaytype
        self.I = 1    
        self.D_h = 1.
        self.D_LSEh=1.
        self.enc_w = enc_w
        #self.drop = np.random.choice(self.Nh_e, int(0.75*self.Nh_e), replace=False) #for dropping a certain amount of neurons in the encoded hidden layer
        
        
        
        ### ENCODING & LEARNING DECODERS###
        
        #Create samples to train decoders with 
        V = V_train(self.Zeta, self.N_batch).T
        #print("V_tr shape:", V.shape)
        #Encoding weights & bias (binary)
        h_rad = max_mag(XX@XX.T)[0]
        if self.enc_w == 'binary':
            self.W_e = h_rad*(2*(np.random.binomial(n=1, p=0.5, size=(self.Nh_e, self.Nh))) - 1) 
            #self.B_e = 2*(np.random.binomial(n=1, p=0.5, size=(self.Nh*self.N_batch, self.Nh_e))) - 1 
        #Encoding weights & bias (real)
#         if self.enc_w == 'real':
#             self.W_e = self.Nv*np.sqrt(self.Nh)*np.random.normal(0, 1, size=(self.Nh, self.Nh_e))               
#             self.B_e = np.random.normal(0, 1, size=(self.Nh_e,))      

        #Activities of hidden layer
        #Ah = self.Zeta@V.T
        #print("\Xi shape:", self.Zeta.shape)
        H = self.Zeta @ V
        #print("act of H:", H.T[:10])
        LSE_h = np.log( np.sum( np.exp(H), axis=0 ) )
        #print('lse(h):', LSE_h[:10])
        
        #Project/encode the hidden layer
        Ah = sigmoid(self.W_e@H)
        #print("shape of act of encoded hidden:", Ah.shape)
        #Want to decode A and LSE(A) from Act (learn the decoders)
        self.D_h = np.linalg.lstsq(Ah.T, H.T, rcond=None)[0].T
        self.D_LSEh = np.linalg.lstsq(Ah.T, LSE_h.T, rcond=None)[0].T
#         print(self.D_LSEh.shape)
#         print('Decoded act of h:', (self.D_h@Ah).T[:10])
#         print('Decoded LSE(h):', self.D_LSEh@Ah)
        
    def pack(self, v, h, f, c, X):
        '''
         z = net.pack(v, h, f, c, X)
         Can be undone using v,h,f,c,X = net.unpack(z).
        '''
        if self.bs==1:
            X=X.ravel().reshape((self.Nh*self.Nv, 1))
            return np.concatenate((v,h,f,c.T,X), axis=0).ravel()
        else:
            c.reshape(1, self.Nh)
            #print('shapes:', v.shape, h.shape, f.shape, c.shape, X.shape)
            return np.concatenate((v,h,f,c,X), axis=0).ravel()
    
    def unpack(self, z):
        '''
         v,he,f,c,X = net.unpack(z)
         Can be undone using net.pack(v, he, f, c, X).
        '''

        Nh = self.Nh
        Nv = self.Nv
        Nh_e = self.Nh_e
        bs = self.bs
        v = z[ : Nv*bs]
        hstart = len(v)
        he = z[hstart : hstart + Nh_e*bs]
        fstart = hstart + len(he)
        f = z[fstart : fstart + Nh*bs]
        cstart = fstart + len(f)
        c = z[cstart : cstart + bs]
        Xstart = cstart + len(c)
        X = z[Xstart :]

        v = v.reshape( (Nv, bs) )
        he = he.reshape( (Nh_e, bs) )
        f = f.reshape( (Nh, bs) )
        X = X.reshape( (Nv, Nh) )
        c = c.reshape( (bs, 1 ) )
        

        
        return v, he, f, c, X
    
    def dynamics(self, t, z, training):
        '''
         z = [v, h, f, c, X]

         During training weights (training = True):
         dv and dh are set to zero (so v and h do not change)
         v and h should be set to the targets and 1-hot respectively by setting their components
         in the ititial z array (y_0) to these values and setting their derivative to zero
         f, c, X, all follow the dynamical equations below.
         @equilibrium, f = h - LSE(h) (Nh x Nh),  c = LSE(h) (Nh x 1), X = Zeta.T (Nh x Nv)


         During testing (training = False):
         dX is set to zero (so X will not change)
         X is fixed to be the weights by setting initial z array (y_0) to these values and setting dXdt to zero.
         v, h, f, c all follow their dynamical equations.
         @equilibrium, f = h - LSE(h) (Nh x Nh),  c = LSE(h) (Nh x 1), v = (1/kappa)*(Zeta.T softmax(h) + I) 
        '''
        Nh = self.Nh
        Nv = self.Nv
        bs = self.bs
    
            
    
        if training: #training
        #if we want to learn the weights, clamp v and h by setting initial values to targets and 1-hot and dvdt=dhdt=0. 

            v, h_e, f, c, X = self.unpack(z) 
#             print('v.shape:', v.shape)
#             print('he.shape:', h_e.shape)
#             print('f.shape:', f.shape)
#             print('c.shape:', c, c.shape)
#             print('X.shape:', X.shape)

            #print(h)
            #h_e = sigmoid(self.W_e@h)
#             print('h_e.shape:', h_e.shape)
#             print('decoders shape', self.D_h.shape, self.D_LSEh.shape)
#             print('decoded h:', (self.D_h@h_e).T)
        
            dv = np.zeros_like(v)
            dh_e = np.zeros_like(h_e)
            df = ( self.D_h@h_e - c.T - f) / self.tau_sm
            dc = ( self.D_LSEh@h_e - c.T ) / self.tau_sm
            dX = ( v @ np.exp(f) - X )/ self.tau_w
            #dX = (np.outer(np.exp(f), v)[0].reshape((Nv, Nh)) - X)/ self.tau_w 
        
        
        
        else: #testing (not training) 
        #if we want to actually use the network, we clamp X to the weights by setting this in the initial values and dXdt=0.
         #this means we will take 1 target at a time, so the dimensions reduce here
            
            v,h_e,f,c,X = self.unpack(z) 
#             v =   z[                 : self.Nv        ]
#             h_e = z[self.Nv          : self.Nv+self.Nh_e]
#             f =   z[self.Nv+self.Nh_e: self.Nv+self.Nh_e+self.Nh]
#             c =   z[-1]
            
            #print('W_e.shape:', self.W_e.shape)
            
            
            dX = np.zeros_like(X)
            dh_e = ( sigmoid( self.W_e@(X.T@v) ) - h_e ) / self.tau_h
            
            #uncomment below to turn off of the neurons in the projected hidden layer (randomly selected)
            #dh_e=[0 if ind in self.drop else x for ind, x in enumerate(dh_e)]
            
            if t<self.t_I: 
                self.I = self.I
                Beta=1
            if t>=self.t_I:
                self.I = 0
                Beta = 0
                
            dv = ( (1-Beta)*(X@np.exp(f)) - self.kappa*v + self.I*(Beta) ) / self.tau_w  

            dc = ( self.D_LSEh@h_e - c )/self.tau_sm
            
            df = ( self.D_h@h_e - c - f ) / self.tau_sm

        
        dzdt = self.pack(dv, dh_e, df, dc, dX)
        return dzdt

### Nearest_Target Experiments 

In [9]:
def Nearest_Targ_Exp(Nh, Nv, runs):
    '''
    Runs LSEHopfield [runs] times for a random binary input. 
    Goal is to show that the network converges to the pattern with the smallest hamming distance to the input.
    Returns the rate at which this occurs
    '''

    closest = 0
    closest_bio = 0
    avg_hamming = 0
    avg_hamming_change = 0 
    
    for i in range(runs):
        print('Run:', i)
        
        ### Create data ###
        X = np.random.binomial(n=1, p=0.5, size=(Nh, Nv))
        XX = 2.*X - 1

        
        
        ### Learn weights ###
        tspan=[0, 3]
        #Initialize network
        net_bio_learn_all=BioLSEHopfield_Learn(XX, beta=1., tau_w=0.01, tau_h=0.01, tau_sm=0.0001, t_I=0, kappa=1., decaytype="Off")
        
        
        
        #Initial state for net
        y0_bio_learn_all = (np.random.random(( (2*Nv) + (2*Nh) + 1 , Nh )) /50.).ravel()
        #Clamp v and h to ideal values
        v = XX.T.ravel()
        h = np.eye(Nh).ravel()
        h=h*20
        y0_bio_learn_all[     : Nv*Nh        ] = v
        y0_bio_learn_all[Nv*Nh: Nv*Nh + Nh*Nh] = h
        
        #Solve network to learn (of side of) the weights
        sol_blearn_all = solve_ivp(net_bio_learn_all.dynamics, tspan, y0_bio_learn_all, method='RK45', args=(True,))
        vf, hf, ff, cf, Xff = net_bio_learn_all.unpack(sol_blearn_all.y.T[-1])
        #X_learned = weight_update(0.5, XX, 10000)
        
        
        
        
        ### RUN ###
        tspan = [0, 2]
        #Initialize network
        #BIO LSE Network
        net_bio = BioLSEHopfield_Learn(Xff.T, beta=10., tau_w=0.05, tau_h = 0.05, tau_sm=0.0001, t_I = 1, kappa=1, decaytype="Off", batchsize=1)
        #ORIGINAL LSE Network 
        net = LSEHopfield(XX, beta=1., tau=0.05, kappa=1., decaytype="Off", split_time=1)
        
        #Initial state for net:
        #For BIO LSE (using learned weights)
        y0_bio_1 = np.random.random((Nv+2*Nh+Nh*Nv+1,)) / 10.
        y0_bio_1[-Nh*Nv:] = Xff.ravel()
        #For OG LSE
        y0 = np.random.random((Nv+Nh,))/10
        
        #Set I to be a random binary input 
        I= np.random.binomial(n=1, p=0.5, size=(Nv,))
        I = 2.*I - 1
        #OR select a pattern and (optionally) perturb it
#         k = np.random.randint(len(XX))
#         II = XX[k] #Correct pattern
#         I = Perturb( XX , n=0, p=0.40 ) #Perturbed the data (binary perturbations)
#         I = I[k]
        net_bio.I=I.reshape( (Nv, 1) )
        
        #How close is this input to all other targets? the network should converge to one of the closest targets
        hams = Hamming([I.T], XX)
        #print('hamming dist between I and patterns:', hams)
        closest_patterns = np.flatnonzero(hams == hams.min())
        
        
        #Solve network 
        sol_bio = solve_ivp(net_bio.dynamics, tspan, y0_bio_1, args=(False,))
        sol = solve_ivp(net.dynamics(I), tspan, y0)
        
        #Store states of network 
        v_bio = sol_bio.y[:Nv,:].T
        vf = sol.y[:Nv].T
        hf = sol.y[Nv:].T
        ##final states
        vf_bio, hf_bio, ff_bio, cf_bio, Xf_bio = net_bio.unpack(sol_bio.y.T[-1])
        
        k_conv_bio = np.argmax(softmax(hf_bio))
        k_conv = np.argmax(softmax(hf[-1]))
        #print('hamming dist between this pattern and the input:', ham(vf, I.reshape((Nv,1))))
        
        if i%25 == 0:
            print('closest patterns to I:', closest_patterns)
            print('with a hamming distance of:', np.min(hams))
            print('BIO LSE network converges to pattern', k_conv_bio)
            print('OG LSE network converges to pattern', k_conv)

            
            #BIO LSE PLOT
            plot_state(sol_bio.t, v_bio, I)
            plt.axvline(x=1, c='grey', linestyle='--')
            plt.axvspan(0, 1, color='0.9')
            plt.title(f'Bio LSE Hopfield Network State Plot', color='black', size=14);
            plt.xlabel('time (s)', color='black', size=12)
            plt.tick_params(axis='x', colors='black')
            plt.tick_params(axis='y', colors='black')
            plt.ylabel('feature neuron (v) states ',color='black', size=12)
            #plt.savefig('stateplt.pdf')
            plt.show()
            #OG LSE PLOT
            plot_state(sol.t, vf, I)
            plt.axvline(x=1, c='grey', linestyle='--')
            plt.axvspan(0, 1, color='0.9')
            plt.title(f'LSE Hopfield Network State Plot', color='black', size=14);
            plt.xlabel('time (s)', color='black', size=12)
            plt.tick_params(axis='x', colors='black')
            plt.tick_params(axis='y', colors='black')
            plt.ylabel('feature neuron (v) states ',color='black', size=12)
            plt.show()

    
        if k_conv_bio in closest_patterns:
            closest_bio += 1
            avg_hamming += np.min(hams)
            avg_hamming_change += ( np.min(hams) - ham(vf[-1], XX[k_conv]) ) 
        if k_conv in closest_patterns:
             closest += 1
        
        #print()
        #print()
           
    return closest_bio/runs, closest/runs, avg_hamming_change/runs, avg_hamming/runs

In [10]:
def Nearest_Targ_ENCODED(Nh, Nv, N_E, Nsample, runs):
    '''
    Runs LSEHopfield [runs] times for a random binary input. 
    Goal is to show that the network converges to the pattern with the smallest hamming distance to the input.
    Returns the rate at which this occurs
    '''

    closest = 0
    avg_hamming = 0
    avg_hamming_change = 0 
    for i in range(runs):
        print('Run:', i)
        
        #Create data
        X = np.random.binomial(n=1, p=0.5, size=(Nh, Nv))
        XX = 2.*X - 1
          
        ### RUN ###
        
        ### Initialize network
        net_e = BioLSEHopfield_Encode(XX, N_E, beta=1., tau_w=0.1, tau_h=0.05, tau_sm=0.0001, t_I=1.0, kappa=1., 
                                      decaytype="Off", enc_w = 'real', N_batch=Nsample)
        #net_LSE = LSEHopfield(XX, beta=1., tau=0.1, kappa=1., decaytype="Off", split_time=1.0)
        
        
        ### Set I to be a random binary input 
        I = np.random.binomial(n=1, p=0.5, size=(Nv,))
        I = 2.*I - 1
        net_e.I=I
        #How close is this input to all other targets? the network should converge to one of the closest targets
        hams = Hamming([I.T], XX)
        closest_patterns = np.flatnonzero(hams == hams.min())
        print('closest patterns to I:', closest_patterns, "with hamming of", np.min(hams))
        

        ### Solve 
        tspan = [0, 3]
        y0=np.random.random((net_e.Nh_e+net_e.Nv+net_e.Nh+1,) )/100
        #y0 = np.random.random( (Nv+Nh,) )/100
        sol = solve_ivp(net_e.dynamics(I), tspan, y0)
        #sol_LSE = solve_ivp(net_LSE.dynamics(I), tspan, y0)
        
        ### Final states
        v_e = sol.y[        :net_e.Nv].T
        h_e = sol.y[net_e.Nv:net_e.Nv+net_e.Nh_e].T[-1]  
        #v = sol_LSE.y[        :Nv].T
        #h = sol_LSE.y[Nv:].T[-1]
        k_conv = np.argmax(softmax(net_e.D_h@h_e))
        #k_conv = np.argmax(softmax(h))
        print('Net identifies pattern number as:', k_conv)
        print('feature neurons stabilize to:', v_e[-1])
        print('correct target pattern:', XX[closest_patterns])
        #print("SM of encoded hidden representation:", softmax(h_e), '\n')
        #print("SM of decoded 1hot representation:", softmax(net_e.W_d@h_e))
        print('Hamming distance between feature neurons and target pattern:', ham(v_e[-1], XX[k_conv]))
        #print('Hamming distance between feature neurons and pattern',k_conv,":",ham(v[-1], XX[k_conv]))

        
      
        
        ### Uncomment below and set runs=1 to get an example plot
        plot_state(sol.t, v_e, I)
        plt.axvline(x=1.0, c='grey', linestyle='--')
        plt.axvspan(0, 1.0, color='0.9')
        plt.title(f'Bio LSE HN (Encoded) State Plot', color='black', size=14);
        plt.xlabel('time (s)', color='black', size=12)
        plt.tick_params(axis='x', colors='black')
        plt.tick_params(axis='y', colors='black')
        plt.ylabel('feature neuron (v) states ',color='black', size=12)
        plt.savefig('stateplt.pdf')
        plt.show()
        
        
        if k_conv in closest_patterns:
            closest += 1
            avg_hamming += np.min(hams)
            avg_hamming_change += ( np.min(hams) - ham(v_e[-1], XX[k_conv]) ) 
            
           
    return closest/runs, avg_hamming_change/runs, avg_hamming/runs

In [11]:
def Nearest_Targ_Encode_Learn(Nh, Nhe, Nv, tI, runs):
    '''
    Runs BioLSE_Encode_Learn [runs] times for a random binary input:
        In each run, a "memory dataset" of Nh targets with Nv bits each is constructed. This nework encodes the hidden representation in Nhe neurons.
        The network learns the weights from the SM subnetwork to the feature neurons (f to v connection, Xi.T). 
        Once these weights are learned we provide a random target as input for tI seconds.
        Once the input is turned OFF we check whether or not the network can fully recover the closest target from the memory dataset 
    
    Goal is to show that the network converges to the target pattern with the smallest hamming distance to the input.
    Returns the rate at which this occurs
    
    '''

    closest = 0
    closest_BE = 0
    avg_hamming = 0
    avg_hamming_change = 0 
    
    for i in range(runs):
        print('Run:', i)
        
        ### Create data ###
        X = np.random.binomial(n=1, p=0.5, size=(Nh, Nv))
        XX = 2.*X - 1
        
        
        ### Learn weights ###
        tspan=[0, 0.5]
        #Initialize network
        net_BE_learn=BioLSE_Encode_Learn(XX, Nh_e = Nhe, beta=1., tau_w=0.1, tau_h=0.01, tau_sm=0.001, t_I=1, kappa=1., decaytype="Off", enc_w = 'binary', N_batch = 500)


        #Initial state for net
        y0_BE_learn = (np.random.random( (2*Nv + Nhe + Nh + 1, Nh) ) / 100).ravel()
        #Clamp v and h to ideal values
        v = XX.T.ravel()
        h = XX@XX.T
        h_E =  (sigmoid(net_BE_learn.W_e@h)).ravel()
        y0_BE_learn[     : Nv*Nh        ] = v
        y0_BE_learn[Nv*Nh: Nv*Nh + Nh*Nhe] = h_E


        #Solve network to learn the weights
        sol_BE_learn = solve_ivp(net_BE_learn.dynamics, tspan, y0_BE_learn, args=(True,))
        vf, hf, ff, cf, Xff = net_BE_learn.unpack(sol_BE_learn.y.T[-1])
        print(Xff.T)
        print(XX)
        
        
        ### RUN ###
        tspan = [0, 2]
        #Initialize network
        
        #BIO LSE Network
        net_BE = BioLSE_Encode_Learn(Xff.T, Nh_e = Nhe, beta=1., tau_w=0.05, tau_h = 0.05, tau_sm=0.0001, t_I = 1, kappa=1, decaytype="Off", enc_w = 'binary', N_batch = 1000, batchsize=1)
        #ORIGINAL LSE Network 
        #net = LSEHopfield(XX, beta=1., tau=0.05, kappa=1., decaytype="Off", split_time=1)
        
        #Initial state for net:
        #For BIO LSE (using learned weights)
        y0_BE_test = np.random.random( (Nv + Nhe + Nh + Nh*Nv + 1,) ) / 10.
        y0_BE_test[-Nh*Nv:] = Xff.ravel()
        #For OG LSE
        #y0 = np.random.random((Nv+Nh,))/10
        
        #Set I to be a random binary input 
        I= np.random.binomial(n=1, p=0.5, size=(Nv,))
        I = 2.*I - 1
        #OR select a pattern and (optionally) perturb it
#         k = np.random.randint(len(XX))
#         II = XX[k] #Correct pattern
#         I = Perturb( XX , n=0, p=0.40 ) #Perturbed the data (binary perturbations)
#         I = I[k]
        net_BE.I=I.reshape( (Nv, 1) )
        
        #How close is this input to all other targets? the network should converge to one of the closest targets
        hams = Hamming([I.T], XX)
        #print('hamming dist between I and patterns:', hams)
        closest_patterns = np.flatnonzero(hams == hams.min())
        
        
        #Solve network 
        sol_BE = solve_ivp(net_BE.dynamics, tspan, y0_BE_test, args=(False,))
        #sol = solve_ivp(net.dynamics(I), tspan, y0)
        
#         #Store states of network 
#         v_bio = sol_bio.y[:Nv,:].T
#         vf = sol.y[:Nv].T
#         hf = sol.y[Nv:].T
        ##final states
        vf_BE, hf_BE, ff_BE, cf_BE, Xf_BE = net_BE.unpack(sol_BE.y.T[-1])
        print(vf_BE)
        print(hf_BE)
        
        
        
        k_conv_BE = np.argmax(softmax(net_BE.D_h@hf_BE))
        #k_conv = np.argmax(softmax(hf[-1]))
        #print('hamming dist between this pattern and the input:', ham(vf, I.reshape((Nv,1))))
        
        if i%25 == 0:
            print('closest patterns to I:', closest_patterns)
            print('with a hamming distance of:', np.min(hams))
            print('BIO LSE network converges to pattern', k_conv_BE)
            #print('OG LSE network converges to pattern', k_conv)

            
            #BIO LSE PLOT
            plot_state(sol_BE.t, vf_BE, I)
            plt.axvline(x=1, c='grey', linestyle='--')
            plt.axvspan(0, 1, color='0.9')
            plt.title(f'Bio LSE HN (Encoded Hidden Rep.) State Plot', color='black', size=14);
            plt.xlabel('time (s)', color='black', size=12)
            plt.tick_params(axis='x', colors='black')
            plt.tick_params(axis='y', colors='black')
            plt.ylabel('feature neuron (v) states ',color='black', size=12)
            #plt.savefig('stateplt.pdf')
            plt.show()
#             #OG LSE PLOT
#             plot_state(sol.t, vf, I)
#             plt.axvline(x=1, c='grey', linestyle='--')
#             plt.axvspan(0, 1, color='0.9')
#             plt.title(f'LSE Hopfield Network State Plot', color='black', size=14);
#             plt.xlabel('time (s)', color='black', size=12)
#             plt.tick_params(axis='x', colors='black')
#             plt.tick_params(axis='y', colors='black')
#             plt.ylabel('feature neuron (v) states ',color='black', size=12)
#             plt.show()

    
        if k_conv_BE in closest_patterns:
            closest_BE += 1
            avg_hamming += np.min(hams)
            avg_hamming_change += ( np.min(hams) - ham(vf[-1], XX[k_conv]) ) 
#         if k_conv in closest_patterns:
#              closest += 1
        
        #print()
        #print()
           
    return closest_BE/runs, avg_hamming_change/runs, avg_hamming/runs

### TESTING
Can we learn the weights using the BIO LSE network with the encoded hidden rep.?

In [12]:
X = np.random.binomial(n=1, p=0.5, size=(5, 4))
XX = 2.*X - 1

Nh, Nv = XX.shape
Nh_E = 500
### Learn weights ###
tspan=[0, 1]
#Initialize network
net_BE_learn=BioLSE_Encode_Learn(XX, Nh_e = Nh_E, beta=1., tau_w=0.01, tau_h=0.0005, tau_sm=0.0001, 
                                 t_I=1, kappa=1., decaytype="Off", enc_w = 'binary', N_batch = 50)


#Initial state for net
y0_BE_learn = (np.random.random( (2*Nv + Nh_E + Nh + 1, Nh) ) / 100).ravel()
#Clamp v and h to ideal values
v = XX.T.ravel()
h = XX@XX.T
h_E =  (sigmoid(net_BE_learn.W_e@h)).ravel()
y0_BE_learn[     : Nv*Nh        ] = v
y0_BE_learn[Nv*Nh: Nv*Nh + Nh*Nh_E] = h_E

#Solve network to learn the weights
sol_BE_learn = solve_ivp(net_BE_learn.dynamics, tspan, y0_BE_learn, args=(True,))
vf, hf, ff, cf, Xff = net_BE_learn.unpack(sol_BE_learn.y.T[-1])
print(Xff.T)
print(XX)

[[-0.9943 -0.8052 -0.6101  0.9978]
 [ 0.8912  0.9265  0.9590  0.9989]
 [-0.9829 -0.9660  0.8577  0.9996]
 [-0.9829 -0.9660  0.8577  0.9996]
 [-0.9709  0.7161 -0.9094  1.0014]]
[[-1.0000 -1.0000 -1.0000  1.0000]
 [ 1.0000  1.0000  1.0000  1.0000]
 [-1.0000 -1.0000  1.0000  1.0000]
 [-1.0000 -1.0000  1.0000  1.0000]
 [-1.0000  1.0000 -1.0000  1.0000]]


In [40]:
v = XX.T
h = (XX@XX.T)
h2 = np.eye(Nh)
h2 = h2*200
print(v, v.shape)
print(h2)

[[-1.0000  1.0000 -1.0000 -1.0000 -1.0000]
 [-1.0000  1.0000 -1.0000 -1.0000  1.0000]
 [-1.0000  1.0000  1.0000  1.0000 -1.0000]
 [ 1.0000  1.0000  1.0000  1.0000  1.0000]] (4, 5)
[[ 200.0000  0.0000  0.0000  0.0000  0.0000]
 [ 0.0000  200.0000  0.0000  0.0000  0.0000]
 [ 0.0000  0.0000  200.0000  0.0000  0.0000]
 [ 0.0000  0.0000  0.0000  200.0000  0.0000]
 [ 0.0000  0.0000  0.0000  0.0000  200.0000]]


In [41]:
c = np.log(np.sum(np.exp(h2[0]), axis = 0)) 
print(c, c.shape)
f = (h2 - c)
print(h2, f)
print(np.exp(f))
X_l = v @ np.exp(f)
X = XX.T
print(X_l)
print(X)

200.0 ()
[[ 200.0000  0.0000  0.0000  0.0000  0.0000]
 [ 0.0000  200.0000  0.0000  0.0000  0.0000]
 [ 0.0000  0.0000  200.0000  0.0000  0.0000]
 [ 0.0000  0.0000  0.0000  200.0000  0.0000]
 [ 0.0000  0.0000  0.0000  0.0000  200.0000]] [[ 0.0000 -200.0000 -200.0000 -200.0000 -200.0000]
 [-200.0000  0.0000 -200.0000 -200.0000 -200.0000]
 [-200.0000 -200.0000  0.0000 -200.0000 -200.0000]
 [-200.0000 -200.0000 -200.0000  0.0000 -200.0000]
 [-200.0000 -200.0000 -200.0000 -200.0000  0.0000]]
[[ 1.0000  0.0000  0.0000  0.0000  0.0000]
 [ 0.0000  1.0000  0.0000  0.0000  0.0000]
 [ 0.0000  0.0000  1.0000  0.0000  0.0000]
 [ 0.0000  0.0000  0.0000  1.0000  0.0000]
 [ 0.0000  0.0000  0.0000  0.0000  1.0000]]
[[-1.0000  1.0000 -1.0000 -1.0000 -1.0000]
 [-1.0000  1.0000 -1.0000 -1.0000  1.0000]
 [-1.0000  1.0000  1.0000  1.0000 -1.0000]
 [ 1.0000  1.0000  1.0000  1.0000  1.0000]]
[[-1.0000  1.0000 -1.0000 -1.0000 -1.0000]
 [-1.0000  1.0000 -1.0000 -1.0000  1.0000]
 [-1.0000  1.0000  1.0000  1.0000 